<a href="https://colab.research.google.com/github/ArmandDS/lexical_simplification_bert/blob/master/Lexical_Simplification_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wordfreq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 4.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from nltk import word_tokenize
from functools import lru_cache
import re
import unicodedata
import sys
from collections import Counter
import nltk
nltk.download('brown')
from nltk.corpus import brown
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.metrics import f1_score
import keras.callbacks
from keras import backend as K
'''from keras.models import Model, Input
from keras import backend as K
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.preprocessing.sequence import pad_sequences'''
import seaborn as sns
import pickle


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'

Mounted at /content/gdrive


In [ ]:
from tensorflow.keras.models import load_model
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM
from nltk import pos_tag
from wordfreq import zipf_frequency
import numpy as np
import pickle

model_cwi = load_model('gdrive/My Drive/model_CWI_full.h5')

with open('gdrive/My Drive/train.pickle', 'rb') as f:
    pickle1 = pickle.load(f)
word2index = pickle1['word2index']  
sent_max_length  = pickle1['sent_max_length']

# Now, let´s define some useful functions in order to use the CWI with some out of samples sentences
# Function for clean the data and remove non characters symbols
stop_words_ = set(stopwords.words('english'))
def cleaner(word):
  #Remove links
  '''word = re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*', 
                '', word, flags=re.MULTILINE)'''
  word = re.sub('[\W]', ' ', word)
  word = re.sub('[^a-zA-Z]', ' ', word)
  return word.lower().strip()

# Function for to create the padded sequence
def process_input(input_text):
  input_text = cleaner(input_text)
  clean_text = []
  index_list =[]
  input_token = []
  index_list_zipf = []
  for i, word in enumerate(input_text.split()):
    if word in word2index:
      clean_text.append(word)
      input_token.append(word2index[word])
    else:
      index_list.append(i)
  input_padded = pad_sequences(maxlen=sent_max_length, sequences=[input_token], padding="post", value=0)
  return input_padded, index_list, len(clean_text)

def complete_missing_word(pred_binary, index_list, len_list):
  list_cwi_predictions = list(pred_binary[0][:len_list])
  for i in index_list:
    list_cwi_predictions.insert(i, 0)
  return list_cwi_predictions

# Second part: The Candidates generation and selection using BERT
# Load the BERT model for masked languge


bert_model = 'bert-large-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model)
model = BertForMaskedLM.from_pretrained(bert_model)
model.eval()  

zipf_frequency('stop', 'en')
zipf_frequency('thwart', 'en')

# Now the function to get the candidates out of BERT (MLM):
def get_bert_candidates(input_text, list_cwi_predictions, numb_predictions_displayed = 10):
  list_candidates_bert = []
  for word,pred  in zip(input_text.split(), list_cwi_predictions):
    if (pred and (pos_tag([word])[0][1] in ['NNS', 'NN', 'VBP', 'RB', 'VBG','VBD' ]))  or (zipf_frequency(word, 'en')) <3.1:
      replace_word_mask = input_text.replace(word, '[MASK]')
      text = f'[CLS]{replace_word_mask} [SEP] {input_text} [SEP] '
      tokenized_text = tokenizer.tokenize(text)
      masked_index = [i for i, x in enumerate(tokenized_text) if x == '[MASK]'][0]
      indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
      segments_ids = [0]*len(tokenized_text)
      tokens_tensor = torch.tensor([indexed_tokens])
      segments_tensors = torch.tensor([segments_ids])
      # Predict all tokens
      with torch.no_grad():
          outputs = model(tokens_tensor, token_type_ids=segments_tensors)
          predictions = outputs[0][0][masked_index]
      predicted_ids = torch.argsort(predictions, descending=True)[:numb_predictions_displayed]
      predicted_tokens = tokenizer.convert_ids_to_tokens(list(predicted_ids))
      list_candidates_bert.append((word, predicted_tokens))
  return list_candidates_bert


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def Convert(string):
    li = list(string.split(" "))
    return li

In [ ]:
def is_empty_or_blank(msg):
    """ This function checks if given string is empty
     or contain only shite spaces"""
    return re.search("^\s*$", msg)

In [ ]:
import pandas as pd
header_list =[ "Paper_Titlef", "Paper_Title", "Pragus", "T5", "facebook"]
#header_list = ["Full_Paper_XML", "Paper_Title","KeyWord", "MeSH", "Eureka-title", "Eureka_sum"]
df = pd.read_csv("gdrive/My Drive/wClustProcessSUMMARY44.csv", sep='\t', names=header_list)
ccc=df.Paper_Title.tolist()
txtp=df.facebook.tolist()
ccc = ccc[104:250]
txtp = txtp[104:250]


In [ ]:
import csv
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import nltk
nltk.download('punkt')
from nltk import tokenize
wordli = []
wordlii =[]
for i, val in enumerate(txtp):
  cccc = ccc[i]
  print('paper name ', cccc, i)
  wordli.append(cccc)
  wordlii.append(cccc)
  new_text2 = []
  input_text2 = []
  yty = tokenize.sent_tokenize(val)

  #print(yty)
  for input_text in yty:
    input_text = cleaner(input_text)
    new_text = input_text
    #print(new_text)
    input_padded, index_list, len_list = process_input(input_text)
  
    pred_cwi = model_cwi.predict(input_padded)
    pred_cwi_binary = np.argmax(pred_cwi, axis = 2)
    complete_cwi_predictions = complete_missing_word(pred_cwi_binary, index_list, len_list)
    #print('complete_cwi_predictions', complete_cwi_predictions)
    '''
    rep = [ '1' if w in bnb else '0' for w in Convert(new_text) ]
    converted_list = []

    for element in rep:
      converted_list.append(element.strip())
    complete_cwi_predictions = converted_list
    #converted_list#[0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]

    #print("complete_cwi_predictions", complete_cwi_predictions)'''
    bert_candidates =   get_bert_candidates(input_text, complete_cwi_predictions)
    print(len(bert_candidates))
    if len(bert_candidates)>1:
     #print(len(bert_candidates))
     for ind in range(len(bert_candidates)):

      #print("bert_candidates", bert_candidates[ind][0])
      wordli.append(bert_candidates[ind][0])
    
    for word_to_replace, l_candidates in bert_candidates:
      tuples_word_zipf = []

      l_candidates =[re.sub('[^a-zA-Z0-9]+', '', _) for _ in l_candidates]

      result = all([is_empty_or_blank(elem) for elem in l_candidates])
      if result:
        l_candidates = [new_text]
      #print(l_candidates,'l_candidates')
      for w in l_candidates:
        if w.isalpha():
          tuples_word_zipf.append((w, zipf_frequency(w, 'en')))
      tuples_word_zipf = sorted(tuples_word_zipf, key = lambda x: x[1], reverse=True)
      if tuples_word_zipf == []:
        #print("same",new_text)
        wordlii.append(new_text)
        new_text = re.sub(word_to_replace, new_text, new_text)
        #print("same",new_text) 
      else:
        #print("tuples_word_zipf", tuples_word_zipf[0][0])
        wordlii.append(tuples_word_zipf[0][0])
        new_text = re.sub(word_to_replace, tuples_word_zipf[0][0], new_text)
        
    data = pd.DataFrame([wordli,wordlii]) #Each list would be added as a row
    data = data.transpose() #To Transpose and make each rows as columns
    data.columns=['Names','Simplify'] #Rename the columns
  data.to_csv("gdrive/My Drive/wClustProcessSUMMARYfacebookcwiList24.csv" , sep='\t', encoding='utf-8', doublequote=False, index=False, header=None, mode="a", quoting=csv.QUOTE_NONE)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


paper name  Combined Genetic and Genealogic Studies Uncover a Large BAP1 Cancer Syndrome Kindred Tracing Back Nine Generations to a Common Ancestor from the 1700s 0
1/1 [==============================] - 1s 1s/step
2
1/1 [==============================] - 0s 55ms/step
6
1/1 [==============================] - 0s 44ms/step
5
1/1 [==============================] - 0s 51ms/step
3
1/1 [==============================] - 0s 46ms/step
6
1/1 [==============================] - 0s 46ms/step
5
1/1 [==============================] - 0s 44ms/step
4
1/1 [==============================] - 0s 44ms/step
4
1/1 [==============================] - 0s 52ms/step
0
1/1 [==============================] - 0s 52ms/step
2
1/1 [==============================] - 0s 45ms/step
1
1/1 [==============================] - 0s 44ms/step
4
1/1 [==============================] - 0s 43ms/step
3
1/1 [==============================] - 0s 44ms/step
4
1/1 [==============================] - 0s 44ms/step
1
1/1 [======================

In [ ]:
len(wordli)

132

In [ ]:
import os
!pip install flashtext

import re
import pandas as pd 
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree
from nltk.tokenize import sent_tokenize

import glob
import re
import pandas as pd
import csv
from os.path import isfile
from flashtext import KeywordProcessor
from pyparsing import StringEnd, oneOf, FollowedBy, Optional, ZeroOrMore, SkipTo

PLURALE_TANTUMS ={'meatus'}
IRREGULAR_NOUNS = {'cornu': 'cornua','pons':'pontes','vas':'vasa','epididymis':'epididymides','femoris':'femora',
'iris':'irides','corpus': 'corpora','plexus': 'plexuses','arthritis':'arthritides', 'crus':'crura','embryo':'embryos',
'femur':'femora','genius':'geniuses','index':'indexes','os':'ossa','paries':'parietes','sinus':'sinuses','virus':'viruses',
'viscus':'viscera'}
def plural(word):
    """
    Converts a word to its plural form.
    """
    if word in PLURALE_TANTUMS:
        # defective nouns, meatus etc
        return word
    elif word in IRREGULAR_NOUNS:
        # IRREGULAR_NOUNS 'cornu': 'cornua','pons':'pontes','vas':'vasa' etc
        return IRREGULAR_NOUNS[word]
    elif word.endswith('us'):
        # Alveolus, alveoli
        return word[:-2] + 'i'
    elif word.endswith('is'):
        # Metastasis, metastases
        return word[:-2] + 'es'
    elif word.endswith('oma'):
        # Carcinoma, carcinomata
        return word + 'ta'
    elif word.endswith('ma'):
        # 
        return word+ 'ta'
    elif word.endswith('a'):
        # vertebra becomes vertebrae.
        return word + 'e'
    elif word.endswith('um'):
        # Acetabulum, acetabula
        return word[:-2] + 'a'
    elif word.endswith("'s"):
        # Acetabulum, acetabula
        return word[:-2]
    elif word.endswith(('yx', 'ax','ix')):
        # Appendix, appendices
        return word[:-1]+ 'ces'
    elif word.endswith(('ex')):
        # Appendix, appendices
        return word[:-2]+ 'ices'
    elif word.endswith('nx'):
        # Larynx, larynges
        return word[:-1]+ 'ges'
    elif word.endswith('x'):
        # Thorax, Thoraces
        return word[:-1]+ 'ces'
    else:
        return word
endOfString = StringEnd()

header_list = ["prefix", "text"]
df2 = pd.read_csv("gdrive/My Drive/prefixFinal3e.csv", sep='\t', names=header_list) 
header_list2 = ["suffix", "word"]
df1 = pd.read_csv("gdrive/My Drive/suffixFinal3.csv", sep='\t', names=header_list2) 
header_list3 = ["root", "term"]
df3 = pd.read_csv("gdrive/My Drive/rootFinal33.csv", sep='\t', names=header_list3) 
prefix = oneOf('ante anti con com extra extro hemi semi hyper hyp hypo intro macro micro micr post pre pro retro bio trans dys hydro intra null per prim secund para polio quadr sub tri ultra anterior posterior necro brady exo pan tachy cephalo caudo endo proximal distal superior inferior mbi ambo amph ana apo aut abdomin aden adren alveol angi arter arteriol athr atri audi balan bronch bronchiol carcin card cellul cerebell cerebr chol cholecyst choledoch chondr chrom col colp cost cry crypt cutane cyan cyst cyst cyt dipl duoden encephal enter epis erythr esophag fibr galact gastr glyc gynec hemat hepat hepatic heter duct fasc flex leiomy myocard myos pronat rhabdomy sarc supinat tens card hidr hist hist hom home hydr hyster iatr ile acetabul calcane carp clavic clavicul clavicle cran femor fibul humer ili isch lumb malleol mandibul maxill intestin jejun kerat lacrima laryng leuk lipid lith lymph melan men mening metr myel myring nat necr nephr neur oophor orchid orchi oss ost palat path peritone pharmac pharyng phleb phren pleur pneum poli proct pulmon pyel rect ren sacr salping scapul sept splen spondyl stern tend ten testicul therm thorac thym thyr thyroid tonsill trache tympan urin ureter urethr vas ven vesic viscer xanth xer blephar cephal cervic cheil chil derm dermat dors gingiv inguin irid lapar lingu mamm mast odont onych ophthalm ocul optic opt pelv pod rhin stomat cheir chir ephr epithel lip myo organ anter caud dist infer later medi poster proxim super ventr kyph lamin lord oste orth scol vertebr metacarp metatars olecran patell phalang pub radi tars tibi uln arthr articul burs disk fibros menisc synov tendin adip albin diaphor erythem myc papill pil rhytid sclera seb squam steat trich aque conjunctiv core cor corne dacry emmetr glauc lacrim mydr phac phak phot presby pupill retin scler uve vitre acous acou aur bar cerumin mastoid staped bas eosin granul hem immun kary lymphaden lymphang neutr mon morph nucle spher sider thromb adenoid alveol atel bronch epiglott nas spir appendic bucc cec celi cheiol bil dent gluc labi gloss pancreat pylor sigmoid submaxill uvul acr calc albumin bacter cortic dips glomerul andr amni chor cyes embry fet fimbr gravida lact mult null vagin vulv umbilic uter omphal perine prim pseud puerper epididym orch prostat sperm spermat test vesicul crin meat medull noct olig trigon alges dur ech esthes gli kines mening phas psych thalam ventricul gonad kal natr parathyroid somat toxic bin epi hyper  inter per poly retro  brach canth capit cili faci gnath occipit phall steth tal trachel ventr dia dis ect ecto end ent ento infra mal mes meso meta uni neo supra sym syn')
suffix = oneOf("algia apheresis ary ase blast capnia centesis chalasis continence cusis cyesis cytosis desis drome ectasis ectomy emia flux gen genesis globin globulin gram graph graphy iasis ician ism itis lithiasis lysis lytic malacia megaly metrist metry ology oma opia opsy optosis orrhaphy orrhea ory oscopy ostomy otomy oxia para paresis pathy pepsia pexy phagia phonia phoresis plasty plegia pnea poiesis prandial ptosis rrhagia rrhaphy rrhea salpinx schisis scope scopy stasis stenosis thenia thorax tocia tresia tripsy tropia uria genic ologist phobia dynia cele cle cule ule ulus cyte emesis ist esis ity osis sis form oid meter stomy penia pexis phagy plasia rrhage rrhexis sclerosis spasm tome trophic trophy plasm sarcoma") + FollowedBy(endOfString)
Dicts = dict(zip(df1['suffix'].astype('str'),df1['word'].astype('str')))
Dictp = dict(zip(df2['prefix'].astype('str'),df2['text'].astype('str')))
Dictr = dict(zip(df3['root'].astype('str'),df3['term'].astype('str')))



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


'gbv = "The main types of cardiomyopathy include dilated, hypertrophic and restrictive cardiomyopathy. Treatment — which might include medications, surgically implanted devices, heart surgery or, in severe cases, a heart transplant — depends on the type of cardiomyopathy and how serious it is."\ngbv = "Myocarditis is inflammation of the heart muscle (myocardium). The inflammation can reduce the heart\'s ability to pump blood. Myocarditis can cause chest pain, shortness of breath, and rapid or irregular heart rhythms (arrhythmias)."\nprint(\'jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj\')\n#gbv1 = gbv.lower()\ntokenized = nltk.word_tokenize(gbv)\nnounss = [word for (word, pos) in nltk.pos_tag(tokenized) if(pos[:1] == \'N\')]\n#print(fname)\nDict2={}\ncleaned2 = \'\'\n#print("\nDictionary with the use of Integer Keys: ") \n#print(Dict)\nword = (ZeroOrMore(prefix)("prefixes") + \n    SkipTo(suffix | endOfString)("root") + \n    Optional(suffix)("suffix"))\nfor wd in noun

In [ ]:
import os
!pip install flashtext

import re
import pandas as pd 
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk import RegexpParser
from nltk import Tree
from nltk.tokenize import sent_tokenize

import glob
import re
import pandas as pd
import csv
from os.path import isfile
from flashtext import KeywordProcessor
from pyparsing import StringEnd, oneOf, FollowedBy, Optional, ZeroOrMore, SkipTo
import pandas as pd

header_list =["Paper_Title", "new1", "old"]
#header_list = ["Full_Paper_XML",, "Pper_Title","KeyWord", "MeSH", "Eureka-title", "Eureka_sum"]
df = pd.read_csv("gdrive/My Drive/wClustProcessSUMMARYfacebookTagWordAll.csv",  sep = "\t", names=header_list)
clust = []
titlo = []
for index, row in df.iterrows():
 wordli = []
 wordlii =[]
 ccc = row['Paper_Title']
 gbv = row['old']
 cleaned2 = ''
 #gbv = f.read()
 #gbv1 = gbv.lower()
 tokenized = nltk.word_tokenize(gbv)
 nounss = [word for (word, pos) in nltk.pos_tag(tokenized) if(pos[:1] == 'N')]
 #print(fname)
 Dict2={}
 cleaned2 = ''
 #print("\nDictionary with the use of Integer Keys: ") 
 #print(Dict)
 word = (ZeroOrMore(prefix)("prefixes") + 
    SkipTo(suffix | endOfString)("root") + 
    Optional(suffix)("suffix"))
 for wd in nounss:
    #print(wd)
     res = word.parseString(wd)
     #print(res.dump())
     rv=res.prefixes
     #print(res.root)
     rvs=res.suffix
     rvr=[res.root]
     k = []
     if len(rvs) != 0:
      #k=[wd]
      for p in rv:
        #print('pre'+p)
        if p in Dictp: 
          value = Dictp[p]
          k.append(value)
      for r in rvr:
        #print('mmmm '+r)
        if r in Dictr: 
          valuer = Dictr[r]
          k.append(valuer)
        elif r.endswith(('o')):
          # Appendix, appendices
          rr = r[:-1]
          if rr in Dictr: 
           valuerr = Dictr[rr]
           #print('newroot '+rr)
           k.append(valuerr)
        else:
           k.append(r)
      for v in rvs:
        #print('mmmm '+p)
        if v in Dicts: 
          value2 = Dicts[v]
          #print('suffix '+v)
          k.append(value2)
      h= " ".join(k)
      #print(h)
      Dict2[wd]=h
 print(Dict2)
 print('Done2')  
 processor = KeywordProcessor(case_sensitive=False)
 for k, v in Dict2.items():
    processor.add_keyword(k, v)
    vcc1 = gbv.lower()
    cvb1 = processor.replace_keywords(vcc1)
    cvb=cvb1.lower()
    vcc2 = cvb
    print('kkk',k)
    print('vv',v)
    wordli.append(k)
    wordlii.append(v)
    sentences = sent_tokenize(vcc2)
    #print('sentenbces', sentences)

    duplicates = list(set([s for s in sentences if sentences.count(s) > 1]))
    #print('duplicates', duplicates)

    cleaned2 = list(set(sentences))
    #print('cleaneds', cleaned2)
 data = pd.DataFrame([wordli,wordlii]) #Each list would be added as a row
 data = data.transpose() #To Transpose and make each rows as columns
 data.columns=['Names','Simplify'] #Rename the columns
 data.to_csv("gdrive/My Drive/wClustProcessSUMMARYfacebooktagListMedTCS.csv" , sep='\t', encoding='utf-8', doublequote=False, index=False, header=None, mode="a", quoting=csv.QUOTE_NONE) #dfhj.to_csv("gdrive/My Drive/wClustProcessSUMMARYfacebookTaggoldmedtcs.csv" , sep='\t', encoding='utf-8', doublequote=False, index=False, header=None, mode="a", quoting=csv.QUOTE_NONE)



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


Streaming output truncated to the last 5000 lines.
kkk lymphocyte
vv lymph vessels cell
{'stability': 'stabil state of', 'instability': 'instabil state of', 'radioactivity': 'radius lower arm bone oactiv state of', 'nitrogen': 'producing'}
Done2
kkk stability
vv stabil state of
kkk instability
vv instabil state of
kkk radioactivity
vv radius lower arm bone oactiv state of
kkk nitrogen
vv producing
{'University': 'Univers state of', 'Laboratory': 'Laborat pertaining to', 'release': 'rele enzyme', 'excitability': 'excitabil state of', 'vibratome': 'vibra instrument'}
Done2
kkk University
vv Univers state of
kkk Laboratory
vv Laborat pertaining to
kkk release
vv rele enzyme
kkk excitability
vv excitabil state of
kkk vibratome
vv vibra instrument
{'diversity': 'divers state of', 'increase': 'incre enzyme', 'neuritis': 'nerve  inflammation', 'cycle': 'cy small', "'Microbiomeanalysis": "'Microbiomeana loosening, separating", 'andchroniccorticosteroneadministrationcausesgutdysbiosis': 'andchr